# Kryptographie

Geheimnisse werden bevorzugt verschlüsselt übermittelt.
Dieses Kapitel zeigt einige Varianten, 
wie einfache Verschlüsselungsmethoden implementiert werden können.

## Caesarkodierung

Die [Caesarkodierung](http://de.wikipedia.org/wiki/Caesar-Verschl%C3%BCsselung) geht auf einfaches Vertauschen von Buchstaben zurück.

In [2]:
from string import uppercase
start = ord("A")
end = ord("Z")
length = end - start

ImportError: cannot import name 'uppercase'

In [3]:
def create_code(n):
    def encoder(word):
        cipher = []
        for char in word:
            if char in uppercase:
                enc = chr((ord(char) - start + n) % length + start)
                cipher.append(enc)
            else:
                cipher.append(char)
        return ''.join(cipher)
    return encoder

In [4]:
code1 = create_code(11)
cc = code1("DAS IST GEHEIM")
cc

NameError: name 'uppercase' is not defined

In [5]:
code2 = create_code(-11)
code2(cc)

NameError: name 'cc' is not defined

Das Problem ist, die Häufigkeit der einzelnen Buchstaben ist nicht gleich.
Daher lässt sich mit einer einfachen Statistik erahnen,
welcher Buchstabe wie verschlüsselt ist.

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
# use a counter, greate a plt.hbar plot, and set the y_axis labels to uppercase

## Enigma

Die [Enigma Verschlüsselungsmaschine](http://en.wikipedia.org/wiki/Enigma_machine) ist ähnlich einer Schreibmaschine.
Buchstaben von "A" bis "Z" werden gedrückt,
um variabel verdrahteten Stromkreise zu schließen.
Nach jedem Tastendruck drehen die Rotoren um einen Schritt weiter.
Je nach der Stellung der Rotoren (und einem fixen Steckbrett) leuchten Lämpchen von "A" bis "Z" auf.

Im folgenden programmieren wir eine an die Enigma Maschine angelehnte Verschlüsselungsmethode in Python.
Zur besseren Lesbarkeit inkludieren wir auch das Leerzeichen, und alle anderen Zeichen werden ohne Verschlüsselung als "." wiedergegeben.
Das Steckbrett wird ausgelassen, es würde nur zusätzlich noch ein paar Buchstaben untereinander austauschen.

Die Funktionsweise ist die folgende:
Die Rotoren und der Reflektor haben intern eine zufällige Permutation gespeichert.
Der initiale Buchstabenindex wird durch die Rotoren zum Permutieren geschickt,
der starre Reflektor dreht den "Stromfluss" um,
und in umgekehrter Reihenfolge die inversen Permutationsoperationen der Rotoren abzuarbeiten.

Nach jedem Buchstaben springt das Zählwerk der Rotoren um eins weiter.

Zur Entschlüsselung müssen die Rotoren der Enigma wieder in ihren Anfangszustand zurückgesetzt werden.

Anzumerken ist außerdem, dass in der echten Version nur eine kleine Anzahl von Rotor und Reflektor Permutationen existierten. Diese Implementation hier erlaubt jegliche Art von Permutationen, eben auch solche, welche einen Kurzschluss im Stromkreis erzeugen würden.

Eine wahrheitsgetreuere Implementation liefert [py-enigma](http://py-enigma.readthedocs.org/en/latest/)

In [8]:
class Rotor(object):
    """
    Der Rotor permutiert den eingegebenen Buchstaben entsprechend seiner Rotationsstellung.
    Nach einer vollen Rotation wird an den nächsten Rotor das Signal zum Vorrücken weitergegeben.
    """
    def __init__(self, length):
        self.length = length
        self.permutation = range(length)
        from random import shuffle
        shuffle(self.permutation)
        self.position = 0
        self.next_rotor = None

    def rotate(self):
        self.position = (self.position + 1) % self.length
        if self.position == 0:
            self.next_rotor.rotate()
            
    def reset(self):
        self.position = 0

    def forward(self, c):
        idx = (self.position + c) % self.length
        return self.permutation[idx]
    
    def inverse(self, c):
        idx = self.permutation.index(c)
        return (idx - self.position) % self.length

In [9]:
class Reflector(Rotor):
    """
    Am Ende der Reihe von Rotoren, gibt es einen Reflector (Unterklasse von Rotor).
    Er schickt das Signal an eine permutierte Stelle zurück für die inverse Operation.
    
    Es gab Varianten mit einem rotierendne Reflektor, hier ist er starr.
    """
    def __init__(self, l):
        super(Reflector, self).__init__(l)
    
    def rotate(self):
        pass
    
    def reset(self):
        pass

    def reflect(self, c, encrypt):
        if encrypt:
            return self.permutation[c]
        else:
            return self.permutation.index(c)

In [10]:
class Enigma(object):
    """
    Diese Enigma Hauptklasse instanziert die Rotoren und den Reflektor.
    Die Methode `run` simuliert den Stromfluss durch die Rotoren und den Reflektor.
    """
    def __init__(self):
        import string
        self.letters = list(string.uppercase + " ")
        l = len(self.letters)
        
        self.rotors = rotors = [Rotor(l), Rotor(l), Rotor(l), Rotor(l)]
        self.reflector = Reflector(l)
        
        for i in range(len(rotors) - 1):
            rotors[i].next_rotor = rotors[i+1]
    
    def reset(self):
        for r in self.rotors:
            r.reset()
        self.reflector.reset()
        
    def run(self, char, encrypt = True):
        char = char.upper()
        if char not in self.letters:
            return "."
        c = self.letters.index(char)
        for r in self.rotors:
            c = r.forward(c)
        c = self.reflector.reflect(c, encrypt)
        for r in self.rotors[::-1]:
            c = r.inverse(c)
        self.rotors[0].rotate()
        return self.letters[c]

In [11]:
import random
seed = 1938 # damit die Zufallsresultate immer gleich sind.
random.seed(seed)
e = Enigma()

AttributeError: module 'string' has no attribute 'uppercase'

In [12]:
from __future__ import print_function
plain = """\
DIES IST EIN GEHEIMER TEXT DEN DIE PROGRAMMIERSPRACHE \
PYTHON IN EINER AN DIE MECHANIK DER ENIGMA CHIFFRIERMASCHINE \
ANGELEHNTEN WEISE VERSCHUESSELN UND ENTSCHLUESSLEN KANN\
"""
e.reset()
cipher = "".join(e.run(char) for char in plain)
print(cipher)

NameError: name 'e' is not defined

In [13]:
e.reset()
for char in cipher:
    print(e.run(char, encrypt=False), end="")

NameError: name 'e' is not defined